# Imports

In [53]:
import spacy
from spacy.tokens.doc import Doc
import pandas as pd
from collections import Counter, defaultdict
from datetime import datetime

In [8]:
nlp = spacy.load('en')

In [9]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'

# Read data

In [7]:
train = pd.read_csv('../data/train.csv').fillna('')
test = pd.read_csv('../data/test.csv').fillna('')
print train.shape, test.shape

(404290, 6) (2345796, 3)


# Write training questions
In the order they appear, not the order of their ids

In [46]:
# with open(train_loc, 'wb') as file_:
#     for idx, question in enumerate(train_array):
#         if idx % 10000 == 0: print(idx, question)
#         file_.write(nlp(question.decode('utf-8')).to_bytes())
with open(train_loc, 'wb') as file_:
    for i in range(len(train)):
        f = train.loc[i]
        if i % 10000 == 0: print i, f['question2']
        file_.write(nlp(f['question1'].decode('utf-8')).to_bytes())
        file_.write(nlp(f['question2'].decode('utf-8')).to_bytes())

0 What is the step by step guide to invest in share market?
10000 Is it safe for a woman to travel alone in Japan?
20000 What triggers you the most when you play video games?
30000 Is domino's pizza halal?
40000 How should I start learning Python for Data Science?
50000 Why is cloning considered unethical?
60000 How do you get rid of a virus on an iPhone?
70000 What are some teenage cake ideas?
80000 How do I recover deleted files on an Android phone or tablet?
90000 Which are the best Hollywood movies of 2016?
100000 What is the process of getting a surgical residency in UK after completing MBBS from India?
110000 How do I become an entrepreneur?
120000 How can I become a billionaire?
130000 What is it like to grow up in Japan, SIngapore and Hong Kong?
140000 Which is a good car in the range of 2-3lacs on Delhi road?
150000 How do I grow my hair really fast and easiest way?
160000 What are the career options after graduating with a B.A. in philosophy?
170000 Is welfare a citizenship r

# Read training data
Elements $(2n, 2n+1)$ are the questions for pair $n$

In [10]:
docs = []
i = 0
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%10000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
#         if i == 10: break
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(type(docs[0]))

2017-05-14 02:34:48
0 2017-05-14 02:34:48
10000 2017-05-14 02:34:51
20000 2017-05-14 02:34:52
30000 2017-05-14 02:34:53
40000 2017-05-14 02:34:54
50000 2017-05-14 02:34:55
60000 2017-05-14 02:34:56
70000 2017-05-14 02:34:58
80000 2017-05-14 02:34:59
90000 2017-05-14 02:35:00
100000 2017-05-14 02:35:01
110000 2017-05-14 02:35:02
120000 2017-05-14 02:35:03
130000 2017-05-14 02:35:04
140000 2017-05-14 02:35:05
150000 2017-05-14 02:35:06
160000 2017-05-14 02:35:07
170000 2017-05-14 02:35:08
180000 2017-05-14 02:35:08
190000 2017-05-14 02:35:09
200000 2017-05-14 02:35:10
210000 2017-05-14 02:35:11
220000 2017-05-14 02:35:12
230000 2017-05-14 02:35:13
240000 2017-05-14 02:35:14
250000 2017-05-14 02:35:15
260000 2017-05-14 02:35:16
270000 2017-05-14 02:35:17
280000 2017-05-14 02:35:18
290000 2017-05-14 02:35:19
300000 2017-05-14 02:35:20
310000 2017-05-14 02:35:21
320000 2017-05-14 02:35:22
330000 2017-05-14 02:35:24
340000 2017-05-14 02:35:25
350000 2017-05-14 02:35:26
360000 2017-05-14 02:3

In [23]:
# print(docs[23986])
# for word in docs[23986]:
#     print(word.lemma_)
    
d = docs[23990]
print([(word.orth_, word.lemma_ for word in d])
# print(Counter([(token.orth_, token.pos_, token.lemma_) for token in d]))

[u'I', u'am', u'financially', u'stuck', u'in', u'a', u'half', u'baked', u'relationship', u'.', u'Can', u'someone', u'help', u',', u'how', u'can', u'I', u'get', u'out', u'of', u'this', u'?']


In [60]:
for i in range(0,len(docs),2):
    d1 = docs[i]
    d2 = docs[i+1]
    c = Counter([(word.pos_, word.lemma_) for word in d1 if not word.is_stop])
    c.subtract(Counter([(word.pos_, word.lemma_ )for word in d2]))
    same = defaultdict(int)
    diff = defaultdict(int)
    for k, v in [(k[0], 1) for k in c.keys() if c[k] == 0]:
        same[k] +=v
    samenum = sum(same.values())
    print same, samenum

# 
    for k, v in [(k[0], abs(c[k])) for k in c.keys() if abs(c[k]) > 0]:
        diff[k] +=v
    diffnum = sum(diff.values())
    print diff, diffnum

    print 
    if i > 10: break
    

defaultdict(<type 'int'>, {u'PUNCT': 1, u'VERB': 1, u'NOUN': 4}) 6
defaultdict(<type 'int'>, {u'DET': 1, u'VERB': 1, u'PART': 1, u'NOUN': 2, u'ADP': 2}) 7

defaultdict(<type 'int'>, {u'PUNCT': 4, u'PROPN': 3}) 7
defaultdict(<type 'int'>, {u'ADV': 1, u'NOUN': 4, u'ADP': 1, u'PROPN': 2, u'DET': 2, u'VERB': 3, u'ADJ': 1}) 14

defaultdict(<type 'int'>, {u'PUNCT': 1, u'VERB': 1, u'NOUN': 1}) 3
defaultdict(<type 'int'>, {u'ADV': 1, u'NOUN': 2, u'ADP': 2, u'PROPN': 2, u'PRON': 1, u'VERB': 3}) 11

defaultdict(<type 'int'>, {}) 0
defaultdict(<type 'int'>, {u'ADV': 2, u'NOUN': 2, u'ADP': 1, u'DET': 1, u'PUNCT': 3, u'VERB': 4, u'NUM': 1, u'ADJ': 1}) 15

defaultdict(<type 'int'>, {u'PUNCT': 1, u'NOUN': 2}) 3
defaultdict(<type 'int'>, {u'NOUN': 5, u'ADP': 1, u'PUNCT': 2, u'VERB': 3, u'X': 1, u'ADJ': 2}) 14

defaultdict(<type 'int'>, {u'PUNCT': 1, u'PROPN': 1}) 2
defaultdict(<type 'int'>, {u'NOUN': 5, u'ADP': 2, u'PRON': 2, u'PROPN': 3, u'DET': 2, u'PUNCT': 5, u'VERB': 3, u'CCONJ': 1, u'ADJ': 2}) 25

# Write test questions

In [54]:
print len(test)
with open(test_loc, 'wb') as file_:
    for i in range(len(test)):
        if i % 10000 == 0: print i
        f = test.loc[i]
        file_.write(nlp(f['question1'].decode('utf-8')).to_bytes())
        file_.write(nlp(f['question2'].decode('utf-8')).to_bytes())

NameError: name 'test' is not defined

# Read test questions

In [7]:
test_docs = []
i = 0
with open(test_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        test_docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
print(type(test_docs[0]))

<type 'spacy.tokens.doc.Doc'>


In [ ]:
test_docs[1345288]

In [ ]:
testloc = 'test_serialize.bin'
with open(testloc, 'wb') as file_:
    file_.write(nlp(u'This is a document.').to_bytes())
    file_.write(nlp(u'This is another.').to_bytes())

In [7]:
docs = []
with open(testloc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        docs.append(Doc(nlp.vocab).from_bytes(byte_string))
assert len(docs) == 2

In [8]:
p = docs[0]

In [12]:
bow = []
for word in p:
#     if word.is_stop: continue
    bow.append(word.lemma_)
Counter(bow)

Counter({u'.': 1, u'a': 1, u'be': 1, u'document': 1, u'this': 1})

In [17]:
print(test.loc[20])

test_id                                                     20
question1    How matter at MIT? Will performing poorly in 1...
question2    I have passed 5 AP tests with scores trump 5. ...
Name: 20, dtype: object


In [24]:
doc = docs[24890]
print doc
print(' '.join(word.lemma_ for word in doc))

What are some good books to read about Islamic philosophy?
what be some good book to read about islamic philosophy ?


In [25]:
lemmas = []
for idx, doc in enumerate(docs):
    lemma = []
    for word in doc:
        if not word.is_punct: lemma.append(word.lemma_)
    lemmas.append(lemma)
    if idx == 20: break

In [ ]:
print(type(lemmas[0][0]))